In [203]:
import ast
import astor
import inspect

In [204]:
class Node(object):
    def __init__(self, obj):
        self.obj = obj

In [233]:
# convert string source code to ast
tree = ast.parse("def f(x): return 2 * (x ** 2) + 1", mode='exec')

In [234]:
# quick view of the entire ast
for statement in tree.body:
    print(ast.dump(statement), '\n')

FunctionDef(name='f', args=arguments(args=[arg(arg='x', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=BinOp(left=Num(n=2), op=Mult(), right=BinOp(left=Name(id='x', ctx=Load()), op=Pow(), right=Num(n=2))), op=Add(), right=Num(n=1)))], decorator_list=[], returns=None) 



In [235]:
binop_stack = []
data_stack = []

# start from root
root = tree.body[0].body[0].value
binop_stack.append(root)
curr = root.left

In [236]:
ast.dump(root.op)

'Add()'

In [237]:
while len(binop_stack) > 0:

    print("evaluating node: ", ast.dump(curr), " \n")

    # scenario 1: if it is a name node
    if isinstance(curr, ast.Name):

        # if its preceding op is a pow
        if isinstance(binop_stack[-1].op, ast.Pow):

            # then we loop at the op so we can grab its right child, which is the exponent
            curr = binop_stack.pop()
            exponent = curr.right.n

            # decrement exponent by 1
            curr.right.n = curr.right.n - 1

            # multiply coefficient by exponent
            curr = data_stack.pop() 
            curr.n = curr.n * exponent

            # new curr
            curr = binop_stack[-1].right

    # scenario 3: if it is a binop    
    elif isinstance(curr, ast.BinOp):
        print(">> adding binop to binop stack", curr.op, "\n")
        binop_stack.append(curr)
        curr = curr.left

    else:
        print(">> adding to data stack", curr.n, "\n")
        data_stack.append(curr)
        curr = binop_stack.pop().right


    print("-" * 20)
    print("data stack", data_stack)
    print("binop stack", binop_stack)
    print("-" * 20)
    print("next node to evaluate", curr)
    print("=" * 40)

evaluating node:  BinOp(left=Num(n=2), op=Mult(), right=BinOp(left=Name(id='x', ctx=Load()), op=Pow(), right=Num(n=2)))  

>> adding binop to binop stack <_ast.Mult object at 0x10859ce10> 

--------------------
data stack []
binop stack [<_ast.BinOp object at 0x10b2b8ac8>, <_ast.BinOp object at 0x10b2b8be0>]
--------------------
next node to evaluate <_ast.Num object at 0x10b2b8438>
evaluating node:  Num(n=2)  

>> adding to data stack 2 

--------------------
data stack [<_ast.Num object at 0x10b2b8438>]
binop stack [<_ast.BinOp object at 0x10b2b8ac8>]
--------------------
next node to evaluate <_ast.BinOp object at 0x10b2b86d8>
evaluating node:  BinOp(left=Name(id='x', ctx=Load()), op=Pow(), right=Num(n=2))  

>> adding binop to binop stack <_ast.Pow object at 0x1085a20f0> 

--------------------
data stack [<_ast.Num object at 0x10b2b8438>]
binop stack [<_ast.BinOp object at 0x10b2b8ac8>, <_ast.BinOp object at 0x10b2b86d8>]
--------------------
next node to evaluate <_ast.Name object

In [214]:
curr = tree.body[0]  
print(astor.to_source(curr)) 

source = astor.to_source(curr)

def f(x):
    return 4 * x ** 1 + 1



In [215]:
# https://stackoverflow.com/questions/48759838/how-to-create-a-function-object-from-an-ast-functiondef-node
code = compile(tree, 'hello.py', 'exec')
namespace = {}
exec(code, namespace)

In [216]:
namespace["f"](1)

5